In [1]:
## USE for Multi GPU Systems
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K


target_names = ["tap", "twotap", "swipeleft", "swiperight", "swipeup", "swipedown", "twoswipeup", "twoswipedown", "circle", "arrowheadleft", "arrowheadright", "checkmark", "flashlight", "l", "lmirrored", "screenshot", "rotate"]

Using TensorFlow backend.


In [2]:
df = pd.read_pickle("DataStudyCollection/df_lstm_norm50.pkl")

lst = df.userID.unique()
np.random.seed(42)
np.random.shuffle(lst)
test_ids = lst[-5:]
train_ids = lst[:-5]
print(train_ids, test_ids)
df.TaskID = df.TaskID % 17

x = np.concatenate(df.Blobs.values).reshape(-1,50,27,15,1)
x = x / 255.0

# convert class vectors to binary class matrices (one-hot notation)
num_classes = len(df.TaskID.unique())
y = utils.to_categorical(df.TaskID, num_classes)

labels = sorted(df.TaskID.unique())

[ 1  2  9  6  4 14 17 16 12  3 10 18  5] [13  8 11 15  7]


In [3]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
#config = tf.ConfigProto(device_count = {"GPU": 1})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allow_growth=True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.3

# set session config
sess = tf.Session(config=config)
tf.keras.backend.set_session(sess)

In [4]:
model = tf.keras.models.load_model('./ModelSnapshots/LSTM-v2-00398.h5')

In [5]:
%%time
lst = []
batch = 100
for i in range(0, len(x), batch):
    _x = x[i : i+batch]
    lst.extend(model.predict(_x))

CPU times: user 47.2 s, sys: 6.47 s, total: 53.6 s
Wall time: 30.9 s


In [6]:
len(df)

9193

In [7]:
df["TaskIDPred"] = lst
df.TaskIDPred = df.TaskIDPred.apply(lambda x: np.argmax(x))

In [8]:
df_train = df[df.userID.isin(train_ids)]
df_test = df[df.userID.isin(test_ids)]

In [9]:

print(sklearn.metrics.confusion_matrix(df_train.TaskID.values, df_train.TaskIDPred.values, labels=labels))
cm = sklearn.metrics.confusion_matrix(df_train.TaskID.values, df_train.TaskIDPred.values, labels=labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(np.round(cm,1))
print("Accuray: %.3f" % sklearn.metrics.accuracy_score(df_train.TaskID.values, df_train.TaskIDPred.values))
print("Recall: %.3f" % metrics.recall_score(df_train.TaskID.values, df_train.TaskIDPred.values, average='macro'))
#print("Precision: %.2f" % metrics.average_precision_score(df_train.TaskID.values, df_train.TaskIDPred.values))
print("F1-Score: %.3f" % metrics.f1_score(df_train.TaskID.values, df_train.TaskIDPred.values, average='macro'))
print(sklearn.metrics.classification_report(df_train.TaskID.values, df_train.TaskIDPred.values, target_names=target_names))

[[323   0   0   0   0   0   0   0   0   0   1   0   1   0   0   0  11]
 [  1 443   0   0   0   0   1   0   0   0   0   0   0   0   0   0   0]
 [  1   0 333   1   0   0   0   0   0   0   0   0   0   0   2   0   0]
 [  3   0   0 317   0   0   0   0   0   0   0   0   0   1   0   0   0]
 [  1   0   0   0 321   1   0   1   0   0   0   0   1   0   0   0   0]
 [  1   0   0   0   0 327   0   0   0   0   0   0   1   1   0   1   3]
 [  0   0   0   0   0   0 431   2   0   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0   1   1 436   0   0   0   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   0 396   1   0   3   0   1   2   0   0]
 [  1   1   6   1   0   0   0   0   0 374   0   3   0   2   0   0   0]
 [  0   0   0   4   0   1   0   0   0   0 380   0   0   0   1   2   0]
 [  0   0   0   0   0   0   0   0   0   0   0 364   0   0   0   0   0]
 [  0   0   0   2   4   2   0   0   0   0   0   0 394   0   0   0   1]
 [  0   0   0   6   0   3   0   0   1   3   0   4   1 383   0   0   0]
 [  0 

In [10]:
print(sklearn.metrics.confusion_matrix(df_test.TaskID.values, df_test.TaskIDPred.values, labels=labels))
cm = sklearn.metrics.confusion_matrix(df_test.TaskID.values, df_test.TaskIDPred.values, labels=labels)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print(np.round(cm,1))
print("Accuray: %.3f" % sklearn.metrics.accuracy_score(df_test.TaskID.values, df_test.TaskIDPred.values))
print("Recall: %.3f" % metrics.recall_score(df_test.TaskID.values, df_test.TaskIDPred.values, average='macro'))
#print("Precision: %.2f" % metrics.average_precision_score(df_test.TaskID.values, df_test.TaskIDPred.values))
print("F1-Score: %.3f" % metrics.f1_score(df_test.TaskID.values, df_test.TaskIDPred.values, average='macro'))
print(sklearn.metrics.classification_report(df_test.TaskID.values, df_test.TaskIDPred.values, target_names=target_names))

[[ 98   5   1   1   0   7   0   5   0   0   0   0   5   0   0   0   4]
 [  2 130   0   0   4   0   6   1   0   0   0   0   4   0   0   0   1]
 [  0   0 103   0   0   0   0   0   0   0   0   0   0   0  27   0   0]
 [  0   0   0 106   0   0   0   0   0   2   0   1   1  17   0   0   0]
 [  0   0   0   0 116   0   0   0   2   0   0   0  10   0   0   0   0]
 [  0   0   0   0   1 124   0   0   0   0   0   1   2   0   6   0   1]
 [  0   0   0   0  17   0 144   0   0   0   0   0   3   0   0   0   0]
 [  0   0   0   0   0  18   0 151   0   0   0   0   0   0   0   0   0]
 [  0   3   1   0   0   0   1   0 130   8   0   4   0   4  14   1   0]
 [  0   0   1   0   0   0   0   0   0 136   0   3   1  11   0   2   0]
 [  0   0   0   0   0   1   0   0   0   1 124   0   0   0   4  22   0]
 [  0   0   0   2   1   0   0   0   2   1   0 140   1   0   0   0   0]
 [  0   0   1   0   6  15   0   0   1   0   0   0 139   2   0   0   0]
 [  0   0   0   0   0   3   0   0   0   2   0   2   2 149   1   0   0]
 [  1 

# Export 

In [11]:
output_node_prefix = "output_node"
num_output = 1
pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = output_node_prefix+str(i)
    pred[i] = tf.identity(model.outputs[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)
output_node_prefix = pred_node_names[0]

output nodes names are:  ['output_node0']


In [12]:
model.inputs[0]

<tf.Tensor 'time_distributed_10_input:0' shape=(?, 50, 27, 15, 1) dtype=float32>

In [13]:
#sess = K.get_session()

In [14]:
output_path = "./Models/"
output_file = "LSTM.pb"

In [15]:
g = tf.GraphDef()
g.ParseFromString(open(output_path + output_file, "rb").read())
s = ""
for n in g.node:
    s =s + str(n.name) + "\n"

print(s)

time_distributed_10_input
time_distributed_10/kernel
time_distributed_10/kernel/read
time_distributed_10/bias
time_distributed_10/bias/read
time_distributed_10/Reshape/shape
time_distributed_10/Reshape
time_distributed_10/convolution
time_distributed_10/BiasAdd
time_distributed_10/Relu
time_distributed_10/Reshape_1/shape
time_distributed_10/Reshape_1
time_distributed_11/kernel
time_distributed_11/kernel/read
time_distributed_11/bias
time_distributed_11/bias/read
time_distributed_11/Reshape/shape
time_distributed_11/Reshape
time_distributed_11/convolution
time_distributed_11/BiasAdd
time_distributed_11/Relu
time_distributed_11/Reshape_1/shape
time_distributed_11/Reshape_1
time_distributed_12/Reshape/shape
time_distributed_12/Reshape
time_distributed_12/MaxPool
time_distributed_12/Reshape_1/shape
time_distributed_12/Reshape_1
time_distributed_13/Reshape/shape
time_distributed_13/Reshape
time_distributed_13/keras_learning_phase/input
time_distributed_13/keras_learning_phase
time_distribut

In [16]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)

graph_io.write_graph(constant_graph, output_path, output_file, as_text=False)

print('Saved the freezed graph at: ', (output_path + output_file))

INFO:tensorflow:Froze 16 variables.
INFO:tensorflow:Converted 16 variables to const ops.
Saved the freezed graph at:  ./Models/LSTM.pb
